In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
path="/content/drive/Shareddrives/Road/FRCNN Google colab/data_augmentation"
os.chdir(path)
os.listdir(path)

In [ ]:
pip install pascal-voc-writer

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa


def get():
    def sometimes(aug): return iaa.Sometimes(0.5, aug)

    return iaa.Sequential(
        [
         
         

            
        ],
        random_order=True
    )

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import cv2
from pascal_voc_writer import Writer
import xml.etree.ElementTree as ET
import glob
from util import annotation as an
import shutil
import sys
from pathlib import Path

INPUT_DIR = 'img'
OUTPUT_DIR = 'output'
AUGMENT_SIZE = 2


def main():
    for file in glob.glob('%s/*.xml' % INPUT_DIR):
        print('Augmenting %s ...' % file)
        annotation = an.parse_xml(file)
        augment(annotation)
        break

    for file in glob.glob('%s/*.xml' % OUTPUT_DIR):
        an.inspect(file)
        break
        
    xmlpath = []
    imgpath = []
    for idx, i in enumerate(glob.glob(OUTPUT_DIR+"/*")):
      if (i[len(i)-3:]).lower() == "xml":
        xmlpath.append(i)
      if (i[len(i)-3:]).lower() == "jpg" or (i[len(i)-3:]).lower() == "png":
        imgpath.append(i)

    for idx,i in enumerate(xmlpath):
      test = i.split("/")
      test = test[0]+"/xmlfile/"+test[1]
      Path(xmlpath[idx]).rename(test)

    for idx,i in enumerate(imgpath):
      test = i.split("/")
      test = test[0]+"/imgfile/"+test[1]
      Path(imgpath[idx]).rename(test)

def augment(annotation):
    seq = get()

    for i in range(AUGMENT_SIZE):
        filename = annotation['filename']
        sp = filename.split('.')
        outfile = '%s/%s-%02d.%s' % (OUTPUT_DIR, sp[0], i, sp[-1])

        seq_det = seq.to_deterministic()

        image = cv2.imread('%s/%s' % (INPUT_DIR, annotation['filename']))
        _bbs = []
        for obj in annotation['objects']:
            bb = ia.BoundingBox(x1=int(obj['xmin']),
                                y1=int(obj['ymin']),
                                x2=int(obj['xmax']),
                                y2=int(obj['ymax']),
                                label=obj['name'])
            _bbs.append(bb)

        bbs = ia.BoundingBoxesOnImage(_bbs, shape=image.shape)

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes(
            [bbs])[0].remove_out_of_image().cut_out_of_image()

        writer = Writer(outfile,
                        annotation['size']['width'],
                        annotation['size']['height'])
        for bb in bbs_aug.bounding_boxes:
            if int((bb.x2-bb.x1)*(bb.y2-bb.y1)) == 0:
                print("augmentet boundingbox has non existing area. Skipping")
                continue
            writer.addObject(bb.label,
                             int(bb.x1),
                             int(bb.y1),
                             int(bb.x2),
                             int(bb.y2))

        cv2.imwrite(outfile, image_aug)
        writer.save('%s.xml' % outfile.split('.')[0])



if __name__ == '__main__':
    main()


In [ ]:
import glob
from xml.etree import ElementTree

imgList = []
allfile = []

for idx, i in enumerate(glob.glob(OUTPUT_DIR+"/xmlfile/*")):
  allfile.append(i)
  if idx == 10:
    break

prositon = []
for i in range(len(allfile)):
    tree = ElementTree.parse(allfile[i])
    root = tree.getroot()
    filenameDataset = root.findall(".//filename")[0].text
    countObj = root.findall(".//object")
    temp = []
    for j in range(len(countObj)):
        listIndexDataset =     [countObj[j].findall(".//xmin")[0].text,
                                countObj[j].findall(".//ymin")[0].text,
                                countObj[j].findall(".//xmax")[0].text,
                                countObj[j].findall(".//ymax")[0].text]
        classDataset =          countObj[j].findall(".//name")[0].text
        part = filenameDataset
        temp.append([listIndexDataset[0]+","+listIndexDataset[1]+","+listIndexDataset[2]+","+listIndexDataset[3]])
        
    data = {}
    data[part] = temp
    prositon.append(data)
  

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

for i in range(len(prositon)):
  im = Image.open(OUTPUT_DIR+"/imgfile/"+list(prositon[i].keys())[0])
  fig, ax = plt.subplots()
  ax.imshow(im)
  for element in prositon[i][list(prositon[i].keys())[0]]:
    xmin, ymin, xmax, ymax= element[0].split(",")

    X_MIN, Y_MIN = int(xmin), int(ymin)
    X_MAX, Y_MAX = int(xmax), int(ymax)
    WIDTH = X_MAX - X_MIN
    HEIGHT = Y_MAX - Y_MIN

    rect = patches.Rectangle((int(X_MIN), int(Y_MIN)), WIDTH, HEIGHT, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
  plt.show()


In [ ]:
imgList = []
xmlpath = []

for idx, i in enumerate(glob.glob(OUTPUT_DIR+"/xmlfile/*")):
  xmlpath.append(i)
for idx, i in enumerate(glob.glob(OUTPUT_DIR+"/imgfile/*")):
  imgList.append(i)

for idx,i in enumerate(xmlpath):
  Path(xmlpath[idx]).rename("/content/drive/Shareddrives/Road/FRCNN Google colab/FRCNN/dataset/annotations"+"/"+xmlpath[idx].split("/")[2])

for idx,i in enumerate(imgpath):
  Path(imgpath[idx]).rename("/content/drive/Shareddrives/Road/FRCNN Google colab/FRCNN/dataset/images"+"/"+imgpath[idx].split("/")[2])